In [92]:
import pandas as pd
import numpy as np
import json
import requests as r
import time
from tqdm import tqdm

In [110]:
precip = pd.read_json("data_sources/additional_sources/2020_precipitation.json")
temp = pd.read_json("data_sources/additional_sources/2020_temperature.json")

idToLatLon = pd.concat([precip[["id", "latitude", "longitude"]], temp[["id", "latitude", "longitude"]]]) \
    .drop_duplicates() \
    .sort_values(["latitude", "longitude"]) \
    .reset_index(drop = True)

In [111]:
idToLatLon.head()

,id,latitude,longitude
0,AYW00090001,-90.000,0.000
1,AYM00089606,-78.450,106.867
2,AYM00089034,-77.867,-34.617
3,AYM00089022,-75.450,-26.217
4,AYM00089662,-74.700,164.100


In [116]:
# using fcc area api
def latLonRequest(lat, lon): 
    result = r.request(
        'get', 
        'https://geo.fcc.gov/api/census/block/find', 
        params={'latitude':lat, 'longitude': lon, 'censusYear': 2020, 'format': 'json'})
    return json.loads(result.text)

def latLonToCounty(row, lat_column="latitude", lon_column="longitude"): 
    block_response = latLonRequest(row[lat_column], row[lon_column])
    block_response = block_response if block_response is not None else {}
    row["county"] = block_response.get("County", {}).get("name")
    row["county_fips"] = block_response.get("County", {}).get("FIPS")
    row["state_id"] = block_response.get("State", {}).get("code")
    row["state_name"] = block_response.get("State", {}).get("name")
    row["state_fips"] = block_response.get("State", {}).get("FIPS")
    row["api_request_status"] = block_response.get("status")
    return row

In [117]:
latLonRequest(45.0900, -71.2906)

{'Block': {'FIPS': '330079501002038',
  'bbox': [-71.303427, 45.087563, -71.287331, 45.09677]},
 'County': {'FIPS': '33007', 'name': 'Coos'},
 'State': {'FIPS': '33', 'code': 'NH', 'name': 'New Hampshire'},
 'status': 'OK',
 'executionTime': '0'}

In [118]:
# calls need delay since api docs specify need to space requests
def latLonToCountyRequestWithDelay(df, lat_column, lon_column, delay_sec=0.5):
    for (index, data) in tqdm(df.iterrows()):
        time.sleep(delay_sec)
        yield latLonToCounty(data, lat_column, lon_column)

In [ ]:
for i in range(0, (idToLatLon.shape[0] // 1000) + 1): 
    print("part {}".format(i))
    latLonCounties = list(latLonToCountyRequestWithDelay(idToLatLon[i*1000:(i+1)*1000], "latitude", "longitude", 0.5))
    pd.DataFrame(latLonCounties) \
        .to_csv("data_sources/additional_sources/2020_weather_station_to_county_{part}.csv".format(part=i), 
                index=False)

0it [00:00, ?it/s]

part 0


1000it [15:05,  1.10it/s]
0it [00:00, ?it/s]

part 1


368it [05:34,  1.21it/s]

In [99]:
idToLatLon.shape[0]

13695